In [1]:
using LilGuys
using Arya
using CairoMakie
using HDF5

In [4]:
using OrderedCollections

In [34]:
function get_uncertainties(filename)
    local δx, δv
    HDF5.h5open(filename) do f
        δx = LilGuys.get_vector(f, "position_errs")
        δv = LilGuys.get_vector(f, "velocity_errs")
    end

    return δx, δv
end

get_uncertainties (generic function with 1 method)

In [63]:
function compare_orbits(labels_filenames)
    orbits = OrderedDict(label => Orbit(filename) for (label, filename) in labels_filenames)

    labels = [label for (label, _) in labels_filenames]
    filenames = [filename for (_, filename) in labels_filenames]
    uncerts = [get_uncertainties(filename) for filename in filenames]

    
    dx = [u[1] for (label, u) in uncerts]
    dv = [u[2] for (label, u) in uncerts]


    pos = LilGuys.positions.(values(orbits))
    rs = radii.(pos)
    ts = LilGuys.times.(values(orbits))

    
    LilGuys.plot_xyz(pos..., labels=labels) |> display

    plot_deviations(ts, rs, labels) |> display

    plot_deviations(ts, [log10.(x) for x in rs ./ dx], labels, ylabel="log # sigma") |> display

end

compare_orbits (generic function with 1 method)

In [64]:
function plot_deviations(ts, rs, labels; ylabel = "deviation")
    fig = Figure()
    ax = Axis(fig[1,1],
        xlabel = "index",
        ylabel = ylabel
    )


    for (t, r, label) in zip(ts, rs, labels)
        lines!(t, r, label=label)
    end
    
    fig
end

plot_deviations (generic function with 2 methods)

In [67]:
labels = Dict(
    "default" => "1e7_new/fiducial/centres.hdf5",
    "smaller" => "1e7_new/fiducial/centres_r0.3.hdf5",
    "potential cut" => "1e7_new/fiducial/centres_potential.hdf5",
    "most bound" => "1e7_new/fiducial/centres_mb.hdf5",
    "com" => "1e7_new/fiducial/centres_com.hdf5",
    )

Dict{String, String} with 5 entries:
  "com"           => "1e7_new/fiducial/centres_com.hdf5"
  "default"       => "1e7_new/fiducial/centres.hdf5"
  "most bound"    => "1e7_new/fiducial/centres_mb.hdf5"
  "potential cut" => "1e7_new/fiducial/centres_potential.hdf5"
  "smaller"       => "1e7_new/fiducial/centres_r0.3.hdf5"

In [ ]:
compare_orbits(labels)

In [59]:
orbits = OrderedDict(
    "default" => Orbit("1e7_new/fiducial/centres.hdf5"),
    "r0.3" => Orbit("1e7_new/fiducial/centres_r0.3.hdf5"),
    )

OrderedDict{String, Orbit} with 2 entries:
  "default" => Orbit([0.0, 10.0928, 19.9268, 30.0195, 40.1123, 49.9463, 60.0391…
  "r0.3"    => Orbit([0.0, 10.0928, 19.9268, 30.0195, 40.1123, 49.9463, 60.0391…

All in all, the details of centring do not matter too much in isolation, so we can just use the centre. Note that the naieve uncertainties are substantially underrepresented. 

Comparing the methods, the COM is likely the true centre for isolation but does not apply for tidal stripping. As such, the other methods, like Most Bound and SS scatter around the mean